# Exercício 1 - Simulador e Agente Reativo Simples em Ambiente Parcialmente Observável

## Imports
Para começar vamos importar algumas funções que vamos utilizar no código.


In [7]:
import random as r

## Classe: Enviroment
Primeiro fiz uma classe que representa o ambiente que o agente estará inserido. Esta classe implementará as funções que são relacionadas as sujeiras do chão.

In [25]:
class Enviroment:
    FLOOR_STATE = {"clean": 0, "dirty": 1}
    
    def __init__ (self, size: int):
        self.rooms = [0] * size

    def dirty_room(self, position: int):
        self.rooms[position] = Enviroment.FLOOR_STATE["dirty"]
            
    def clean_room(self, position: int):
        self.rooms[position] = Enviroment.FLOOR_STATE["clean"]
                
    def init_randomly(self, dirtyRate: int):
        for i in range(len(self.rooms)):
            if r.randint(0, 100) < dirtyRate:
                self.dirty_room(i)
            else:
                self.clean_room(i)
    
    def init_rooms(self, rooms: list):
        self.rooms = rooms.copy()

    def rand_dirty_rooms (self, dirtyRate: int):
        for i, room in enumerate(self.rooms):
            if(room == Enviroment.FLOOR_STATE["clean"] and r.randint(0, 100) < dirtyRate):
                self.dirty_room(i)

## Classe: Agent
A classe agent representará o agente e possui a função que determina como o mesmo irá se comportar dado o ambiente.

In [64]:
class Agent:
    ACTIONS = {"NoOp": 0, "clean": 2, "right": 1, "left": -1}
    
    def __init__ (self):
        self.position = None
        
    def initial_position(self, initial_position: int):
        self.position = initial_position
    
    def action (self, env: Enviroment):
        if env.rooms[self.position] == Enviroment.FLOOR_STATE["dirty"]:
            env.rooms[self.position] = Enviroment.FLOOR_STATE["clean"]
            
            return Agent.ACTIONS["clean"]
        
        temp = 1 if r.randint(0, 100) < 50 else -1

        if (temp == Agent.ACTIONS["left"] and self.position == 0) or (temp == Agent.ACTIONS["right"] and self.position == len(env.rooms) - 1):
            temp = Agent.ACTIONS["NoOp"]
            
        self.position += temp
            
        return temp

## Classe: Evaluator
A classe evaluator vai representar a avaliação do agente e possui a função de pontuar e definir o tempo de avaliação que ocorre.

In [70]:
class Evaluator:
    def __init__ (self, env: Enviroment, agent: Agent, steps: int, seed: int, dirtyRate: int):
        self.env = env
        self.agent = agent
        self.steps = steps
        self.seed = seed
        self.dirtyRate = dirtyRate
        self.agent_points = 0
    
    def execute (self):
        self.agent_points = 0
        for i in range(self.steps):
            self.evaluate_agent(self.agent.action(self.env))
            self.env.rand_dirty_rooms(self.dirtyRate)
        
        return self.agent_points
        
    def evaluate_agent (self, action: int):
        if action == Agent.ACTIONS["clean"]:
            self.agent_points += 1

In [96]:
env = Enviroment(2)
agent = Agent()
eval = Evaluator(env, agent, 1000, 123, 50)

env_possibilities = [[0, 0], [0, 1], [1, 0], [1, 1]]
agent_possibilities = [0, 1]
result = []

for agent_possibility in agent_possibilities:
    print(f"Agente com posição inicial {agent_possibility}: ")
    for env_possibility in env_possibilities: 
        agent.initial_position(agent_possibility)
        env.init_rooms(env_possibility)
        result.append(eval.execute())
        print(f"\tAmbiente inicial: {env_possibility} | Pontuação: {result[-1]}")
    print()
print(f"Média das pontuações: {sum(result)/len(result)}")

Agente com posição inicial 0: 
	Ambiente inicial: [0, 0] | Pontuação: 569
	Ambiente inicial: [0, 1] | Pontuação: 568
	Ambiente inicial: [1, 0] | Pontuação: 598
	Ambiente inicial: [1, 1] | Pontuação: 590

Agente com posição inicial 1: 
	Ambiente inicial: [0, 0] | Pontuação: 602
	Ambiente inicial: [0, 1] | Pontuação: 594
	Ambiente inicial: [1, 0] | Pontuação: 582
	Ambiente inicial: [1, 1] | Pontuação: 581

Média das pontuações: 585.5
